### **1.- Carga y limpieza DF**

In [1]:
import pandas as pd

import pymysql # Para ver la versión
import sqlalchemy # Para ver la versión
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
host = "rawg-db.c5gics8qchki.eu-north-1.rds.amazonaws.com"
user = "lorenzo"
password = "Lorenzo123"
database = "rawg_db"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")


connection = engine.connect()


connection.close()

In [20]:
sql = """SELECT
  g.*,
  COALESCE(s.store_ids,    ARRAY[]::int[])   AS store_ids,
  COALESCE(ge.genre_ids,   ARRAY[]::int[])   AS genre_ids,
  COALESCE(p.platform_ids, ARRAY[]::int[])   AS platform_ids
FROM public.games g
LEFT JOIN (
  SELECT gs.game_id,
         ARRAY_AGG(DISTINCT gs.store_id ORDER BY gs.store_id) AS store_ids
  FROM public.game_stores gs
  GROUP BY gs.game_id
) s  ON s.game_id = g.game_id
LEFT JOIN (
  SELECT gg.game_id,
         ARRAY_AGG(DISTINCT gg.genre_id ORDER BY gg.genre_id) AS genre_ids
  FROM public.game_genres gg
  GROUP BY gg.game_id
) ge ON ge.game_id = g.game_id
LEFT JOIN (
  SELECT gp.game_id,
         ARRAY_AGG(DISTINCT gp.platform_id ORDER BY gp.platform_id) AS platform_ids
  FROM public.game_platforms gp
  GROUP BY gp.game_id
) p  ON p.game_id = g.game_id
ORDER BY g.game_id;
"""
df_games = pd.read_sql_query(sql, engine)

# ¿Existen juegos que aun no están “con señal” (metacritic o ratings > 0)?

df_games_signal = df_games[(df_games['metacritic'].fillna(0) > 0) | (df_games['rating'].fillna(0) > 0)]
df = df_games_signal
df.shape

(1457, 23)

In [21]:
df=df[['game_id', 'name', 'rating', 'added', 'metacritic', 'genre_ids','store_ids','platform_ids','esrb_rating_id']]

In [22]:
df = df.copy()
df['esrb_rating_id'] = df['esrb_rating_id'].fillna(6)
df['metacritic'] = df['metacritic'].fillna(df['metacritic'].mean())
df['metacritic'] = df['metacritic'].round(2)

In [23]:
df.head(3)

,game_id,name,rating,added,metacritic,genre_ids,store_ids,platform_ids,esrb_rating_id
0,1,D/Generation HD,1.86,128,78.93,"[3, 7]","[1, 2, 3, 6]","[1, 4, 5, 7, 18]",2.0
1,25,Middle-earth: Shadow of War,3.86,8701,82.00,"[4, 5]","[1, 2, 3, 4, 5, 8]","[1, 4, 18, 21]",4.0
2,28,Red Dead Redemption 2,4.59,16533,96.00,[4],"[1, 2, 3, 11]","[1, 4, 18]",4.0


In [24]:
df.isna().sum()

game_id           0
name              0
rating            0
added             0
metacritic        0
genre_ids         0
store_ids         0
platform_ids      0
esrb_rating_id    0
dtype: int64

### **2.- Definir variable Éxito**

Normaliza: metacritic/100, rating/5.

Ajustar por género: calcula p90 de added dentro de cada género y compara cada juego contra ese listón (si no hay género, usa p90 global).

Score: combina (0.4 * crítica) + (0.3 * usuarios) + (0.3 * popularidad ajustada).

Éxito: etiqueta 1 al top 20% por score

In [ ]:
#Aquí tienes una métrica de “éxito” que funciona con tu df (tiene: game_id, name, rating, added, metacritic, genre_ids, store_ids, platform_ids, esrb_rating_id):

#Normaliza: metacritic/100 y rating/5 → [0,1].

#Ajusta popularidad por género: calcula el p90 de added dentro de cada género y compara cada juego contra ese listón (si no hay género, usa p90 global).

#Score: 0.4*crítica + 0.3*usuarios + 0.3*popularidad_ajustada.

#Etiqueta Éxito: 1 para el top 20% por score (o un umbral fijo si tienes pocos juegos).

In [25]:
#1. Normalizaciones básicas
df["_metacritic_norm"] = (df["metacritic"] / 100)
df["_rating_norm"] = (df["rating"] / 5)

In [26]:
#2. Umbral de popularidad por género (p90 de 'added' dentro de cada género)
# Explode por género para poder agrupar

df_added_genero = df[["game_id", "added", "genre_ids"]].explode("genre_ids", ignore_index=True)

# p90 de 'added' por género (solo donde hay género no vacío)
genre_p90 = (df_added_genero.loc[df_added_genero["genre_ids"].notna(), :].groupby("genre_ids")["added"].quantile(0.90))

# Mapear el p90 de cada género a cada fila.
# Si un juego tiene varios géneros, tomamos la mediana del p90 de sus géneros como referencia
df_added_genero["p90_added_genre"] = df_added_genero["genre_ids"].map(genre_p90)

# Volver a nivel juego: mediana del p90 de sus géneros
game_ref_added = (df_added_genero.groupby("game_id")["p90_added_genre"]
                                 .median()
                                 .rename("p90_added_genre_med"))

df = df.merge(game_ref_added, left_on="game_id", right_index=True, how="left")

In [27]:
df.head(3)

,game_id,name,rating,added,metacritic,genre_ids,store_ids,platform_ids,esrb_rating_id,_metacritic_norm,_rating_norm,p90_added_genre_med
0,1,D/Generation HD,1.86,128,78.93,"[3, 7]","[1, 2, 3, 6]","[1, 4, 5, 7, 18]",2.0,0.7893,0.372,4641.0
1,25,Middle-earth: Shadow of War,3.86,8701,82.00,"[4, 5]","[1, 2, 3, 4, 5, 8]","[1, 4, 18, 21]",4.0,0.8200,0.772,6981.1
2,28,Red Dead Redemption 2,4.59,16533,96.00,[4],"[1, 2, 3, 11]","[1, 4, 18]",4.0,0.9600,0.918,8243.8


In [28]:
#3. Popularidad ajustada por género
# added relativo a su referencia de género
eps = 1e-9
df["_added_norm"] = (df["added"] / (df["p90_added_genre_med"]+ eps))

In [29]:
#4. Score compuesto
# Pesos sugeridos:
# - Metacritic (crítica): 0.4
# - Rating (usuarios):    0.3
# - Added ajustado:       0.3
df["_score_exito"] = (  0.4 * df["_metacritic_norm"].fillna(0) +
                        0.3 * df["_rating_norm"].fillna(0) +
                        0.3 * df["_added_norm"].fillna(0)
                        )

In [30]:
#5. Umbral para 'éxito'
# Por defecto marcamos como éxito al top 20% por score (percentil 0.80).
umbral = df["_score_exito"].quantile(0.80)

df["exito"] = (df["_score_exito"] >= umbral).astype(int)

In [32]:
df

,game_id,name,rating,added,metacritic,genre_ids,store_ids,platform_ids,esrb_rating_id,_metacritic_norm,_rating_norm,p90_added_genre_med,_added_norm,_score_exito,exito
0,1,D/Generation HD,1.86,128,78.93,"[3, 7]","[1, 2, 3, 6]","[1, 4, 5, 7, 18]",2.0,0.7893,0.372,4641.00,0.027580,0.435594,0
1,25,Middle-earth: Shadow of War,3.86,8701,82.00,"[4, 5]","[1, 2, 3, 4, 5, 8]","[1, 4, 18, 21]",4.0,0.8200,0.772,6981.10,1.246365,0.933510,1
2,28,Red Dead Redemption 2,4.59,16533,96.00,[4],"[1, 2, 3, 11]","[1, 4, 18]",4.0,0.9600,0.918,8243.80,2.005507,1.261052,1
3,30,WipEout Omega Collection,3.55,1682,85.00,[1],[3],[18],2.0,0.8500,0.710,3945.10,0.426352,0.680906,0
4,32,Destiny 2,3.52,14547,82.00,"[2, 4]","[1, 2, 3, 11]","[1, 4, 18, 171, 186, 187]",3.0,0.8200,0.704,9808.25,1.483139,0.984142,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,1004990,PEAK (2025),4.11,14,78.93,"[3, 4, 51]",[1],[4],6.0,0.7893,0.822,4349.00,0.003219,0.563286,0
2578,1006719,RoboCop: Rogue City - Unfinished Business,3.18,30,78.93,"[3, 4]","[1, 5]",[4],4.0,0.7893,0.636,6296.40,0.004765,0.507949,0
2580,1007056,One-Eyed Likho,3.71,24,78.93,"[3, 51]",[1],[4],6.0,0.7893,0.742,4045.15,0.005933,0.540100,0
2584,1007234,The King is Watching,4.29,22,78.93,"[10, 51]",[1],[4],6.0,0.7893,0.858,3411.65,0.006448,0.575055,0


In [33]:
df.isna().sum()

game_id                 0
name                    0
rating                  0
added                   0
metacritic              0
genre_ids               0
store_ids               0
platform_ids            0
esrb_rating_id          0
_metacritic_norm        0
_rating_norm            0
p90_added_genre_med    19
_added_norm            19
_score_exito            0
exito                   0
dtype: int64

1.  **Desarrollo del Modelo (Data Scientist):**
    - Utilizar los datos de PostgreSQL para entrenar un modelo de Machine Learning (ej. con XGBoost) que prediga si un videojuego será un "éxito".

In [36]:
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report, confusion_matrix

y = df["exito"].astype(int)
has_esrb = "esrb_rating_id" in df.columns
if has_esrb:
    df["esrb_rating_id"] = pd.to_numeric(df["esrb_rating_id"], errors="coerce").fillna(-1).astype(int)

# ---------- 1) Limpiar/asegurar tipos ----------
# Numéricas crudas (evita usar columnas derivadas como _score_exito, _added_norm, etc.)
num_cols = [c for c in ["rating", "added","metacritic"] if c in df.columns]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------- 2) Train/Test split (estratificado) ----------
# (Con una fila por juego, partir por filas es equivalente a partir por IDs)
X_base = df[num_cols + ["genre_ids","store_ids","platform_ids","esrb_rating_id"]]
X_train, X_test, y_train, y_test = train_test_split(X_base, y, test_size=0.30, random_state=42, stratify=y)

# ---------- 3) Multi-hot encoders (fit SOLO en train) ----------
def fit_mlb(train_series):
    mlb = MultiLabelBinarizer(sparse_output=False)
    mat = mlb.fit_transform(train_series)
    # nombres de columnas seguros
    cols = [str(c) for c in mlb.classes_]
    return mlb, mat, cols

def transform_mlb(mlb, series):
    mat = mlb.transform(series)
    cols = [str(c) for c in mlb.classes_]
    return mat, cols

# Genres
mlb_gen, gen_train_mat, gen_labels = fit_mlb(X_train["genre_ids"])
gen_train_df = pd.DataFrame(gen_train_mat, columns=[f"genre_{c}" for c in gen_labels], index=X_train.index)
gen_test_mat, _ = transform_mlb(mlb_gen, X_test["genre_ids"])
gen_test_df = pd.DataFrame(gen_test_mat, columns=[f"genre_{c}" for c in gen_labels], index=X_test.index)

# Stores
mlb_store, store_train_mat, store_labels = fit_mlb(X_train["store_ids"])
store_train_df = pd.DataFrame(store_train_mat, columns=[f"store_{c}" for c in store_labels], index=X_train.index)
store_test_mat, _ = transform_mlb(mlb_store, X_test["store_ids"])
store_test_df = pd.DataFrame(store_test_mat, columns=[f"store_{c}" for c in store_labels], index=X_test.index)

# Platforms
mlb_plat, plat_train_mat, plat_labels = fit_mlb(X_train["platform_ids"])
plat_train_df = pd.DataFrame(plat_train_mat, columns=[f"plat_{c}" for c in plat_labels], index=X_train.index)
plat_test_mat, _ = transform_mlb(mlb_plat, X_test["platform_ids"])
plat_test_df = pd.DataFrame(plat_test_mat, columns=[f"plat_{c}" for c in plat_labels], index=X_test.index)

# ---------- 4) One-hot para ESRB (si existe) ----------
if has_esrb:
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    esrb_train = ohe.fit_transform(X_train[["esrb_rating_id"]])
    esrb_cols = [f"esrb_{c}" for c in ohe.categories_[0].tolist()]
    esrb_train_df = pd.DataFrame(esrb_train, columns=esrb_cols, index=X_train.index)

    esrb_test = ohe.transform(X_test[["esrb_rating_id"]])
    esrb_test_df = pd.DataFrame(esrb_test, columns=esrb_cols, index=X_test.index)
else:
    esrb_train_df = pd.DataFrame(index=X_train.index)
    esrb_test_df = pd.DataFrame(index=X_test.index)

# ---------- 5) Ensamblar matrices finales ----------
X_train_num = X_train[num_cols].fillna(0)
X_test_num  = X_test[num_cols].fillna(0)

X_train_final = pd.concat([X_train_num, gen_train_df, store_train_df, plat_train_df, esrb_train_df], axis=1)
X_test_final  = pd.concat([X_test_num,  gen_test_df,  store_test_df,  plat_test_df,  esrb_test_df],  axis=1)

# Quitar columnas constantes (si las hubiera)
const_cols = [c for c in X_train_final.columns if X_train_final[c].nunique(dropna=False) <= 1]
if const_cols:
    X_train_final = X_train_final.drop(columns=const_cols)
    X_test_final  = X_test_final.drop(columns=[c for c in const_cols if c in X_test_final.columns])

# ---------- 6) Modelo: XGBoost con fallback ----------
try:
    from xgboost import XGBClassifier
    clf = XGBClassifier(
        n_estimators=400,
        learning_rate=0.07,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        random_state=42
    )
except ModuleNotFoundError:
    from sklearn.ensemble import HistGradientBoostingClassifier
    clf = HistGradientBoostingClassifier(
        loss="log_loss",
        max_iter=500,
        learning_rate=0.07,
        max_leaf_nodes=31,
        l2_regularization=1.0,
        random_state=42
    )

clf.fit(X_train_final, y_train)

# ---------- 7) Evaluación ----------
proba = clf.predict_proba(X_test_final)[:, 1]
pred  = (proba >= 0.5).astype(int)

acc = accuracy_score(y_test, pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, pred, average="binary", zero_division=0)
try:
    auc = roc_auc_score(y_test, proba)
except Exception:
    auc = np.nan

print(f"AUC: {auc:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")
print("\nClassification report:\n", classification_report(y_test, pred, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(y_test, pred))

# (Opcional) Top importancias
try:
    importances = pd.Series(getattr(clf, "feature_importances_", None), index=X_train_final.columns)
    if importances is not None and importances.notna().any():
        print("\nTOP 20 features:\n", importances.sort_values(ascending=False).head(20))
except Exception:
    pass

C:\Users\loren\anaconda3\envs\mod4\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) [9] will be ignored
  warnings.warn(


AUC: 0.9938 | Acc: 0.9703 | Prec: 0.9213 | Rec: 0.9318 | F1: 0.9266

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       350
           1       0.92      0.93      0.93        88

    accuracy                           0.97       438
   macro avg       0.95      0.96      0.95       438
weighted avg       0.97      0.97      0.97       438

Confusion matrix:
 [[343   7]
 [  6  82]]
